In [1]:
# ============================================
# 1. Imports y configuración general
# ============================================
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    FunctionTransformer
)
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score
)

from sklearn.model_selection import KFold

RANDOM_STATE = 4
np.random.seed(RANDOM_STATE)

pd.set_option("display.max_columns", 50)
pd.set_option("display.float_format", lambda x: f"{x:.4f}")


In [2]:
# ============================================
# 2. Carga de datos
# ============================================
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

print("Train:", train.shape, " Test:", test.shape)
train.head()


Train: (79800, 21)  Test: (34200, 20)


,id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,41996,7hUhmkALyQ8SX9mJs5XI3D,Love and Rockets,Love and Rockets,Motorcycle,22,211533,False,0.3050,0.8490,9,-10.7950,1,0.0549,0.0001,0.0567,0.4640,0.3200,141.7930,4,goth
1,76471,5x59U89ZnjZXuNAAlc8X1u,Filippa Giordano,Filippa Giordano,"Addio del passato - From ""La traviata""",22,196000,False,0.2870,0.1900,7,-12.0300,0,0.0370,0.9300,0.0004,0.0834,0.1330,83.6850,4,opera
2,54809,70Vng5jLzoJLmeLu3ayBQq,Susumu Yokota,Symbol,Purple Rose Minuet,37,216506,False,0.5830,0.5090,1,-9.6610,1,0.0362,0.7770,0.2020,0.1150,0.5440,90.4590,3,idm
3,16326,1cRfzLJapgtwJ61xszs37b,Franz Liszt;YUNDI,Relajación y siestas,"Liebeslied (Widmung), S. 566",0,218346,False,0.1630,0.0368,8,-23.1490,1,0.0472,0.9910,0.8990,0.1070,0.0387,69.4420,3,classical
4,109799,47d5lYjbiMy0EdMRV8lRou,Scooter,Scooter Forever,The Darkside,27,173160,False,0.6470,0.9210,2,-7.2940,1,0.1850,0.0009,0.3710,0.1310,0.1710,137.9810,4,techno


In [3]:
# ============================================
# 3. Definición de target y columnas base
# ============================================
target_col = "popularity"

# columnas de alta cardinalidad (texto libre / ids)
high_card_cols = ["id", "track_id", "artists", "album_name", "track_name"]

# todas las numéricas (incluye popularity)
numeric_cols_all = train.select_dtypes(include=["int64", "float64"]).columns.tolist()

for col in ["artists", "album_name", "track_name"]:
    print(f"{col}: {train[col].nunique()} valores distintos")


artists: 25775 valores distintos
album_name: 37315 valores distintos
track_name: 55767 valores distintos


In [4]:
# ============================================
# 4. Nuevas features
# ============================================

# 4.1 duración en minutos
for df in [train, test]:
    df["duration_min"] = df["duration_ms"] / 60000.0

# 4.2 interacción energy * danceability
for df in [train, test]:
    df["energy_danceability"] = df["energy"] * df["danceability"]

# 4.3 interacción energy * valence (mide cuán "energético + feliz" es el tema)
for df in [train, test]:
    df["energy_valence"] = df["energy"] * df["valence"]


In [5]:
# ============================================
# 5. Gestión de outliers con IQR capping
# ============================================

numeric_features_for_iqr = [
    col for col in numeric_cols_all
    if col != target_col
]

iqr_bounds = {}
for col in numeric_features_for_iqr:
    q1 = train[col].quantile(0.25)
    q3 = train[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    iqr_bounds[col] = (lower, upper)

def cap_iqr(df, bounds):
    df_cap = df.copy()
    for col, (lower, upper) in bounds.items():
        if col in df_cap.columns:
            df_cap[col] = df_cap[col].clip(lower=lower, upper=upper)
    return df_cap

train = cap_iqr(train, iqr_bounds)
test = cap_iqr(test, iqr_bounds)


In [6]:
# ============================================
# 6. Feature: popularidad promedio por género
# ============================================

genre_mean = train.groupby("track_genre")[target_col].mean()
global_mean = train[target_col].mean()

train["genre_pop_mean"] = train["track_genre"].map(genre_mean).fillna(global_mean)
test["genre_pop_mean"] = test["track_genre"].map(genre_mean).fillna(global_mean)


In [7]:
from sklearn.model_selection import KFold

# ============================================
# Nuevas features basadas en popularidad por grupo (OOF, sin leakage)
# ============================================

def add_pop_mean_feature_oof(train, test, col, target_col, smoothing=10, prefix=None, n_splits=5):
    """
    Crea una feature de popularidad promedio suavizada por grupo (artista, álbum, etc.)
    usando esquema OUT-OF-FOLD para evitar leakage.
    - Cada fila del train recibe un valor calculado SIN usar su propio target.
    - El test usa las estadísticas calculadas sobre TODO el train.
    """
    if prefix is None:
        prefix = col

    global_mean = train[target_col].mean()
    new_col = f"{prefix}_pop_mean"

    # inicializamos con NaN
    train[new_col] = np.nan

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)

    for tr_idx, val_idx in kf.split(train):
        tr = train.iloc[tr_idx]
        val = train.iloc[val_idx]

        stats = tr.groupby(col)[target_col].agg(["mean", "count"])
        stats["enc"] = (
            stats["mean"] * stats["count"] + global_mean * smoothing
        ) / (stats["count"] + smoothing)

        train.loc[val.index, new_col] = val[col].map(stats["enc"])

    # si algún valor quedó sin asignar, usar global_mean
    train[new_col] = train[new_col].fillna(global_mean)

    # para el test usamos stats con TODO el train
    stats_full = train.groupby(col)[target_col].agg(["mean", "count"])
    stats_full["enc"] = (
        stats_full["mean"] * stats_full["count"] + global_mean * smoothing
    ) / (stats_full["count"] + smoothing)

    test[new_col] = test[col].map(stats_full["enc"]).fillna(global_mean)

    return train, test

# 1) Popularidad promedio por artista (OOF)
train, test = add_pop_mean_feature_oof(
    train, test,
    col="artists",
    target_col=target_col,
    smoothing=20,
    prefix="artist",
    n_splits=5
)

# 2) Popularidad promedio por álbum (OOF)
train, test = add_pop_mean_feature_oof(
    train, test,
    col="album_name",
    target_col=target_col,
    smoothing=20,
    prefix="album",
    n_splits=5
)

# 3) Desvío estándar de popularidad por género (esta la podés dejar así)
genre_stats = train.groupby("track_genre")[target_col].agg(["std"])
train["genre_pop_std"] = train["track_genre"].map(genre_stats["std"]).fillna(0.0)
test["genre_pop_std"]  = test["track_genre"].map(genre_stats["std"]).fillna(0.0)


In [8]:
genre_stats = train.groupby("track_genre")[target_col].agg(["std"])
train["genre_pop_std"] = train["track_genre"].map(genre_stats["std"]).fillna(0.0)
test["genre_pop_std"]  = test["track_genre"].map(genre_stats["std"]).fillna(0.0)

print("Nuevas columnas creadas:", "artist_pop_mean", "album_pop_mean", "genre_pop_std")
# ============================================
# Features derivadas de popularidad y recuentos
# ============================================

# 1) Popularidad relativa del artista vs género
for df in [train, test]:
    df["artist_vs_genre"] = df["artist_pop_mean"] - df["genre_pop_mean"]

# 2) Popularidad relativa del álbum vs artista
for df in [train, test]:
    df["album_vs_artist"] = df["album_pop_mean"] - df["artist_pop_mean"]

# 3) Cantidad de temas por artista y por álbum
artist_counts = train["artists"].value_counts()
album_counts  = train["album_name"].value_counts()

for df in [train, test]:
    df["artist_track_count"] = df["artists"].map(artist_counts).fillna(0)
    df["album_track_count"]  = df["album_name"].map(album_counts).fillna(0)

# 4) Versión logarítmica de los conteos (para que no dominen tanto)
for df in [train, test]:
    df["artist_track_count_log"] = np.log1p(df["artist_track_count"])
    df["album_track_count_log"]  = np.log1p(df["album_track_count"])


Nuevas columnas creadas: artist_pop_mean album_pop_mean genre_pop_std


In [9]:
# ============================================
# 7. Target encoding OOF para alta cardinalidad
# ============================================
def target_encode_oof(train, test, col, target_col, n_splits=5, smoothing=10):
    """
    Target encoding out-of-fold para la columna categórica `col`.
    Devuelve train, test con una nueva columna f"{col}_te".
    """
    global_mean = train[target_col].mean()
    
    te_col = f"{col}_te"
    train[te_col] = np.nan
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    for train_idx, val_idx in kf.split(train):
        tr = train.iloc[train_idx]
        val = train.iloc[val_idx]
        
        stats = tr.groupby(col)[target_col].agg(["mean", "count"])
        stats["te"] = (stats["mean"] * stats["count"] + global_mean * smoothing) / (stats["count"] + smoothing)
        
        train.loc[val.index, te_col] = val[col].map(stats["te"])
    
    train[te_col] = train[te_col].fillna(global_mean)
    
    stats_full = train.groupby(col)[target_col].agg(["mean", "count"])
    stats_full["te"] = (stats_full["mean"] * stats_full["count"] + global_mean * smoothing) / (stats_full["count"] + smoothing)
    test[te_col] = test[col].map(stats_full["te"]).fillna(global_mean)
    
    return train, test

# Encoding por artista y álbum
train, test = target_encode_oof(train, test, "artists", target_col)
train, test = target_encode_oof(train, test, "album_name", target_col)


In [10]:
# ============================================
# 8. Selección final de features y preprocesador
# ============================================

drop_cols = high_card_cols + [target_col]

feature_cols = [c for c in train.columns if c not in drop_cols]

numeric_features = train[feature_cols].select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = train[feature_cols].select_dtypes(include=["object", "bool"]).columns.tolist()

print("Num:", numeric_features)
print("Cat:", categorical_features)

numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numeric_features),
        ("cat", categorical_pipeline, categorical_features),
    ]
)


Num: ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'duration_min', 'energy_danceability', 'energy_valence', 'genre_pop_mean', 'artist_pop_mean', 'album_pop_mean', 'genre_pop_std', 'artist_vs_genre', 'album_vs_artist', 'artist_track_count', 'album_track_count', 'artist_track_count_log', 'album_track_count_log', 'artists_te', 'album_name_te']
Cat: ['explicit', 'track_genre']


In [11]:
# ============================================
# 9. Train / Validation split (Holdout)
# ============================================
X = train[feature_cols].copy()
y = train[target_col].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=RANDOM_STATE
)

X_train.shape, X_val.shape


((63840, 30), (15960, 30))

In [12]:
# ============================================
# 10. Función auxiliar: entrenar + buscar hiperparámetros + evaluar
# ============================================
def evaluar_modelo(nombre, estimator, param_grid=None, cv=5):
    """
    Crea un Pipeline(preprocessor + estimator),
    realiza GridSearchCV (si hay param_grid),
    entrena en X_train / y_train y evalua en X_val / y_val.
    Devuelve un dict con métricas y mejores hiperparámetros.
    """
    print(f"\n=== {nombre} ===")
    
    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", estimator)
    ])
    
    if param_grid is not None and len(param_grid) > 0:
        grid = GridSearchCV(
            pipe,
            param_grid=param_grid,
            cv=cv,
            scoring="neg_root_mean_squared_error",
            n_jobs=-1,
            verbose=1
        )
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        best_params = grid.best_params_
        cv_rmse = -grid.best_score_
        print(f"Mejores hiperparámetros: {best_params}")
        print(f"Mejor RMSE CV: {cv_rmse:.4f}")
    else:
        best_model = pipe
        best_model.fit(X_train, y_train)
        best_params = {}
        cv_rmse = np.nan
    
    y_pred = best_model.predict(X_val)
    
    mse = mean_squared_error(y_val, y_pred)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    print(f"RMSE val: {rmse:.4f}")
    print(f"MSE  val: {mse:.4f}")
    print(f"MAE  val: {mae:.4f}")
    print(f"R²   val: {r2:.4f}")
    
    resultados = {
        "modelo": nombre,
        "best_params": best_params,
        "cv_rmse": cv_rmse,
        "rmse_val": rmse,
        "mse_val": mse,
        "mae_val": mae,
        "r2_val": r2,
        "best_estimator": best_model
    }
    return resultados


In [13]:
# ============================================
# 11. Definición de modelos y grids de hiperparámetros
# ============================================

modelos = []

# 1) Regresión Lineal
modelos.append({
    "nombre": "LinearRegression",
    "estimator": LinearRegression(),
    "param_grid": {}
})

# 2) Árbol de Decisión
modelos.append({
    "nombre": "DecisionTreeRegressor",
    "estimator": DecisionTreeRegressor(random_state=RANDOM_STATE),
    "param_grid": {
        "model__max_depth": [None, 10, 20],
        "model__min_samples_leaf": [1, 2, 5]
    }
})

# 3) Random Forest
modelos.append({
    "nombre": "RandomForestRegressor",
    "estimator": RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    "param_grid": {
        "model__n_estimators": [200, 400],
        "model__max_depth": [None, 25, 35],
        "model__min_samples_leaf": [1, 2],
        "model__max_features": ["sqrt"]
    }

})

# 4) Gradient Boosting
modelos.append({
    "nombre": "GradientBoostingRegressor",
    "estimator": GradientBoostingRegressor(random_state=RANDOM_STATE),
    "param_grid": {
        "model__n_estimators": [100, 200],
        "model__learning_rate": [0.05, 0.1],
        "model__max_depth": [2, 3],
        "model__min_samples_leaf": [1, 2]
    }
})

# 5) Red Neuronal MLP
modelos.append({
    "nombre": "MLPRegressor",
    "estimator": MLPRegressor(
        random_state=RANDOM_STATE,
        max_iter=300
    ),
    "param_grid": {
        "model__hidden_layer_sizes": [(64,), (64, 32)],
        "model__alpha": [0.0001, 0.001],
        "model__learning_rate_init": [0.001, 0.01]
    }
})


In [45]:
# ============================================
# 12. Entrenamiento, búsqueda de hiperparámetros y evaluación
# ============================================
resultados = []

for m in modelos:
    res = evaluar_modelo(
        nombre=m["nombre"],
        estimator=m["estimator"],
        param_grid=m["param_grid"],
        cv=5
    )
    resultados.append(res)

resultados_df = pd.DataFrame([{
    "modelo": r["modelo"],
    "cv_rmse": r["cv_rmse"],
    "rmse_val": r["rmse_val"],
    "mse_val": r["mse_val"],
    "mae_val": r["mae_val"],
    "r2_val": r["r2_val"],
    "best_params": r["best_params"]
} for r in resultados])

resultados_df.sort_values("rmse_val", inplace=True)
resultados_df



=== LinearRegression ===
RMSE val: 10.7103
MSE  val: 114.7095
MAE  val: 8.0120
R²   val: 0.7655

=== DecisionTreeRegressor ===
Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [13]:
# ============================================
# 13. Selección del mejor modelo según RMSE
# ============================================
mejor_idx = resultados_df["rmse_val"].idxmin()
mejor_nombre = resultados_df.loc[mejor_idx, "modelo"]
print("Mejor modelo según RMSE de validación:", mejor_nombre)

mejor_dict = [r for r in resultados if r["modelo"] == mejor_nombre][0]
mejor_modelo = mejor_dict["best_estimator"]
mejor_modelo


Mejor modelo según RMSE de validación: RandomForestRegressor


,steps,"[('preprocess', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [14]:
# ============================================
# 14. Entrenamiento final con todo el train y predicción sobre test
# ============================================

X_full = train[feature_cols].copy()
y_full = train[target_col].copy()

# Clonamos el mejor modelo para entrenar de cero con todo el train
from sklearn.base import clone
mejor_modelo_full = clone(mejor_modelo)
mejor_modelo_full.fit(X_full, y_full)

X_test = test[feature_cols].copy()
test_preds = mejor_modelo_full.predict(X_test)

submission = pd.DataFrame({
    "id": test["id"],
    "popularity": test_preds
})

display(submission.head())
print(submission.shape)
print(submission.isna().sum())

submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' generado.")


,id,popularity
0,113186,48.7550
1,42819,14.8250
2,59311,2.9050
3,91368,0.1050
4,61000,26.6700


(34200, 2)
id            0
popularity    0
dtype: int64
Archivo 'submission.csv' generado.


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def probar_rf(n_estimators):
    print(f"\n==== Probando RF con {n_estimators} árboles ====")
    
    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=None,
        min_samples_leaf=1,
        max_features="sqrt",
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    
    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", rf)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    
    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)
    
    print(f"RMSE val: {rmse:.4f}")
    print(f"MAE  val: {mae:.4f}")
    print(f"R²   val: {r2:.4f}")
    
    return pipe, rmse

model_200, rmse_200 = probar_rf(200)   # tu config actual
model_300, rmse_300 = probar_rf(300)   # más árboles
model_400, rmse_400 = probar_rf(400)   # aún más árboles




==== Probando RF con 200 árboles ====
RMSE val: 8.0443
MAE  val: 5.2199
R²   val: 0.8677

==== Probando RF con 300 árboles ====
RMSE val: 8.0443
MAE  val: 5.2199
R²   val: 0.8677

==== Probando RF con 300 árboles ====
RMSE val: 8.0399
MAE  val: 5.2180
R²   val: 0.8679

==== Probando RF con 400 árboles ====
RMSE val: 8.0399
MAE  val: 5.2180
R²   val: 0.8679

==== Probando RF con 400 árboles ====
RMSE val: 8.0429
MAE  val: 5.2158
R²   val: 0.8678
RMSE val: 8.0429
MAE  val: 5.2158
R²   val: 0.8678


In [15]:
# ============================================
# Entrenamiento FINAL del Random Forest ganador
# ============================================

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline

# Definimos el mejor modelo encontrado manualmente
rf_final = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=RANDOM_STATE,
    n_jobs=-1
)

final_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", rf_final)
])

# Entrenamiento con TODO el train (sin validación)
X_full = train[feature_cols].copy()
y_full = train[target_col].copy()

final_model.fit(X_full, y_full)

# Predicción sobre test
X_test = test[feature_cols].copy()
test_preds = final_model.predict(X_test)

# Armamos el archivo de submission
submission = pd.DataFrame({
    "id": test["id"],
    "popularity": test_preds
})

submission.to_csv("submission_rf_400_oof.csv", index=False)

print("Archivo 'submission_rf_400_oof.csv' generado!")
submission.head()


Archivo 'submission_rf_400_oof.csv' generado!


,id,popularity
0,113186,47.1800
1,42819,13.7125
2,59311,1.5375
3,91368,0.2750
4,61000,24.3400


In [16]:
print("Cantidad de features:", len(feature_cols))
print(feature_cols)

print("\nDtypes de las features:")
print(train[feature_cols].dtypes)

print("\nPrimeras filas de las features:")
train[feature_cols].head()


Cantidad de features: 30
['duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'duration_min', 'energy_danceability', 'energy_valence', 'genre_pop_mean', 'artist_pop_mean', 'album_pop_mean', 'genre_pop_std', 'artist_vs_genre', 'album_vs_artist', 'artist_track_count', 'album_track_count', 'artist_track_count_log', 'album_track_count_log', 'artists_te', 'album_name_te']

Dtypes de las features:
duration_ms                 int64
explicit                     bool
danceability              float64
energy                    float64
key                         int64
loudness                  float64
mode                        int64
speechiness               float64
acousticness              float64
instrumentalness          float64
liveness                  float64
valence                   float64
tempo                     float64
time_signature     

,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_min,energy_danceability,energy_valence,genre_pop_mean,artist_pop_mean,album_pop_mean,genre_pop_std,artist_vs_genre,album_vs_artist,artist_track_count,album_track_count,artist_track_count_log,album_track_count_log,artists_te,album_name_te
0,211533,False,0.3050,0.8490,9,-10.7950,1,0.0549,0.0001,0.0567,0.4640,0.3200,141.7930,4,goth,3.5255,0.2589,0.2717,28.8824,33.2653,33.2653,11.8783,4.3829,0.0000,1,1,0.6931,0.6931,33.2653,33.2653
1,196000,False,0.2870,0.1900,7,-12.0300,0,0.0370,0.9300,0.0004,0.0834,0.1330,83.6850,4,opera,3.2667,0.0545,0.0253,24.8592,31.3722,32.6336,14.4003,6.5131,1.2614,7,2,2.0794,1.0986,30.1102,32.0594
2,216506,False,0.5830,0.5090,1,-9.6610,1,0.0362,0.7770,0.1263,0.1150,0.5440,90.4590,4,idm,3.6084,0.2967,0.2769,15.7508,28.5850,33.4431,10.3201,12.8343,4.8581,17,2,2.8904,1.0986,26.5501,33.6048
3,218346,False,0.1630,0.0368,8,-17.5150,1,0.0472,0.9910,0.1263,0.1070,0.0387,69.4420,4,classical,3.6391,0.0060,0.0014,12.3138,33.2653,15.9597,17.6045,20.9515,-17.3056,1,27,0.6931,3.3322,33.2653,10.5517
4,173160,False,0.6470,0.9210,2,-7.2940,1,0.1574,0.0009,0.1263,0.1310,0.1710,137.9810,4,techno,2.8860,0.5959,0.1575,38.8066,26.1119,31.8808,17.2636,-12.6947,5.7690,117,12,4.7707,2.5649,25.3965,31.2216
